# <img src="uni-logo.png" alt="Logo UNI" width=150 hight=300 align="right">


<br><br><br>
<h1><font color="#7F000E" size=4>Minería de Datos (CC442)</font></h1>



<h1><font color="#7F000E" size=6>Teoría: Algoritmo K-Nearest Neighbors (KNN) </font></h1>

<br>
<div style="text-align: right">
<font color="#7F000E" size=3>Yuri Coicca, M.Sc.</font><br>
<font color="#7F000E" size=3>Facultad de Ciencias</font><br>
<font color="#7F000E" size=3>Ciencia de la Computación - UNI</font><br>
</div>

El algoritmo **K-Nearest Neighbors (K-NN)** o **K-Vecinos más Cercanos** es uno de los algoritmos de aprendizaje supervisado más simples e intuitivos. A diferencia de otros modelos, K-NN es un algoritmo de **aprendizaje perezoso (lazy learning)**, lo que significa que no tiene una fase de entrenamiento real; simplemente "memoriza" los datos y realiza los cálculos al momento de la predicción.

Aquí tienes la explicación detallada, el pseudocódigo y su implementación.

---

### 1. ¿Cómo funciona K-NN? (Pasos del algoritmo)

1.  **Elegir el valor de $K$**: Decidir cuántos vecinos cercanos se consultarán (por ejemplo, $K=3$ o $K=5$).
2.  **Calcular la distancia**: Para un nuevo punto (dato de prueba), calcular la distancia entre ese punto y **todos** los demás puntos del conjunto de entrenamiento.
    *   *Distancia Euclidiana* (la más común): $d = \sqrt{\sum (x_i - y_i)^2}$
3.  **Seleccionar los vecinos**: Ordenar las distancias de menor a mayor y seleccionar los $K$ puntos más cercanos.
4.  **Realizar la predicción**:
    *   **Clasificación**: Se hace una votación mayoritaria. El nuevo punto pertenece a la clase más frecuente entre sus $K$ vecinos.
    *   **Regresión**: Se calcula el promedio de los valores de los $K$ vecinos.

---

### 2. Pseudocódigo (Lógica pura)

```text
Entrada: DatosEntrenamiento (X, y), PuntoConsulta (q), Valor K

1. Para cada punto 'p' en DatosEntrenamiento:
    a. Calcular distancia D entre 'p' y 'q' (ej. Euclidiana)
    b. Guardar (D, etiqueta_de_p) en una lista

2. Ordenar la lista de distancias de menor a mayor.

3. Tomar los primeros K elementos de la lista ordenada.

4. Si el problema es Clasificación:
    Retornar la clase más frecuente (moda) de los K elementos.
   Si el problema es Regresión:
    Retornar el promedio (media) de los valores de los K elementos.
```

---

### 3. Implementación Manual en Python (Desde cero)

Esta versión ayuda a entender la mecánica interna antes de usar librerías:

```python
import numpy as np
from collections import Counter

def distancia_euclidiana(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        # En KNN, el entrenamiento es solo guardar los datos
        self.X_train = X
        self.y_train = y

    def predict(self, X_test):
        predicciones = [self._predecir_uno(x) for x in X_test]
        return np.array(predicciones)

    def _predecir_uno(self, x_consulta):
        # 1. Calcular distancias contra todos los puntos de entrenamiento
        distancias = [distancia_euclidiana(x_consulta, x_train) for x_train in self.X_train]
        
        # 2. Obtener los índices de los K vecinos más cercanos
        indices_k_vecinos = np.argsort(distancias)[:self.k]
        
        # 3. Extraer las etiquetas de esos vecinos
        etiquetas_vecinos = [self.y_train[i] for i in indices_k_vecinos]
        
        # 4. Votación mayoritaria
        votos = Counter(etiquetas_vecinos).most_common(1)
        return votos[0][0]

# Ejemplo de uso
X_train = np.array([[1, 2], [2, 3], [3, 3], [6, 5], [7, 8]])
y_train = np.array([0, 0, 0, 1, 1]) # Clase 0 y Clase 1

clf = KNN(k=3)
clf.fit(X_train, y_train)
prediccion = clf.predict([[4, 4]])
print(f"El punto [4, 4] pertenece a la clase: {prediccion}")
```

---

### 4. Relación con KD-Tree y Ball-Tree (Optimización)
Es importante notar que el paso 2 de K-NN (calcular la distancia contra **todos** los puntos) tiene una complejidad $O(n)$, lo cual es muy lento si tienes millones de datos.

*   Si usas **fuerza bruta**, K-NN compara el punto contra todo el dataset.
*   Si configuras K-NN para usar un **KD-Tree** o **Ball-Tree**, el algoritmo ya no compara contra todos, sino que navega por el árbol para encontrar los $K$ vecinos en tiempo $O(\log n)$.

---

### 5. Consideraciones Importantes
1.  **Escalado de datos (Feature Scaling)**: K-NN depende totalmente de las distancias. Si una variable está en el rango [0-1] y otra en [0-1000], la segunda dominará la distancia. Siempre hay que **normalizar o estandarizar** los datos.
2.  **Elección de K**:
    *   Un $K$ muy pequeño (ej. $K=1$) es sensible al ruido (overfitting).
    *   Un $K$ muy grande suaviza demasiado las fronteras (underfitting).
    *   Suele elegirse un número impar para evitar empates en la votación.
3.  **Maldición de la dimensionalidad**: En espacios de muchas dimensiones, todos los puntos parecen estar "lejos" de todos, y K-NN pierde efectividad (aquí es donde estructuras como Ball-Tree ayudan, pero no hacen milagros).

### Implementación Profesional (Scikit-learn)
```python
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Escalar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

# Crear y entrenar modelo (aquí puedes elegir 'kd_tree' o 'ball_tree')
model = KNeighborsClassifier(n_neighbors=3, algorithm='auto') 
model.fit(X_scaled, y_train)

# Predecir
pred = model.predict(scaler.transform([[4, 4]]))
```